In [46]:
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from keras import applications
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

datasetpath = r'C:\Users\SurfacePro4\Desktop\dataset'
batch_size = 32
img_size = 64
model = applications.VGG16(include_top=False, weights='imagenet')

vgg_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split = 0.2)

In [47]:
train_generator = vgg_data_gen.flow_from_directory(
        r'C:\Users\SurfacePro4\Desktop\dataset\training',
        subset="training",
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical', 
        shuffle=False)  

cnn_features = model.predict_generator(train_generator, len(train_generator))

Found 1759 images belonging to 4 classes.


In [48]:
filename = 'cnn_features_training'
with open(filename, 'wb') as f:
          np.save(f, cnn_features)

val_generator = vgg_data_gen.flow_from_directory(
        r'C:\Users\SurfacePro4\Desktop\dataset\validation',
        subset="validation",
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
cnn_features = model.predict_generator(val_generator, len(val_generator))
filename = 'cnn_features_validation'
with open(filename, 'wb') as f:
          np.save(f, cnn_features)

Found 121 images belonging to 4 classes.


In [56]:
from keras.utils.np_utils import to_categorical

datagen_top = ImageDataGenerator(rescale=1. / 255)
    
nb_train_samples = len(train_generator.filenames)
num_classes = len(train_generator.class_indices)

np.save('class_indices.npy', train_generator.class_indices)
with open('cnn_features_training', 'rb') as f:
    train_data = np.load(f)

train_labels = train_generator.classes

train_labels = to_categorical(train_labels, num_classes=num_classes)

nb_validation_samples = len(val_generator.filenames)

with open('cnn_features_validation', 'rb') as f:
    validation_data = np.load(f)
    
validation_labels = val_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy', metrics=['accuracy'])


In [57]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)


history = model.fit(train_data, train_labels,
                    epochs=50,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    callbacks=[early_stopping]
                    )

Train on 1759 samples, validate on 121 samples
Epoch 1/50
1759/1759 [==============================] - 3s 2ms/step - loss: 9.3767 - acc: 0.3337 - val_loss: 3.6764 - val_acc: 0.6777
Epoch 2/50
1759/1759 [==============================] - 1s 623us/step - loss: 6.7909 - acc: 0.4986 - val_loss: 1.6064 - val_acc: 0.8512
Epoch 3/50
1759/1759 [==============================] - 1s 604us/step - loss: 5.3135 - acc: 0.6049 - val_loss: 1.7668 - val_acc: 0.8430
Epoch 4/50
1759/1759 [==============================] - 1s 612us/step - loss: 4.5135 - acc: 0.6515 - val_loss: 2.1237 - val_acc: 0.8182
Epoch 5/50
1759/1759 [==============================] - 1s 626us/step - loss: 4.2374 - acc: 0.6828 - val_loss: 1.4897 - val_acc: 0.8678
Epoch 6/50
1759/1759 [==============================] - 1s 672us/step - loss: 4.0637 - acc: 0.6941 - val_loss: 2.5812 - val_acc: 0.8182
Epoch 7/50
1759/1759 [==============================] - 1s 749us/step - loss: 3.6502 - acc: 0.7203 - val_loss: 1.6512 - val_acc: 0.8512
Epo

{'testing': 0, 'training': 1, 'validation': 2}

In [ ]:
batchsize=32
imgsize=64
softmax
3-layers (256*3)
accuracy = 89